In [89]:
import pandas as pd
import ta

In [90]:
df = pd.read_csv('../BTC_ETH_1week_1m.csv')

In [91]:
symbols = ['ETH']

for symbol in symbols:
    df[f'{symbol}_RSI'] = ta.momentum.rsi(close=df[f'{symbol}-USD_Close'], window=14)

df.dropna(inplace=True)

In [92]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, symbols):
        self.short_unit = None
        self.short_amount = 0
        self.short_price = 0
        self.long_unit = None
        self.long_amount = 0
        self.long_price = 0

        self.crossed_up = False

        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.symbols = symbols
        
        self.buys = []
        self.sells = []
        self.history = [100]
        
        self.wins = 0
        self.losses = 0
        self.ties = 0
        
        self.trading_fee_multiplier = 0.99945      # VIP level 0, paying fees with BNB 0.99975
        self.number_of_buys = len(self.buys)

    def buy_long(self, symbol, long_price, time):
        self.balance_amount = self.balance_amount * self.trading_fee_multiplier
        self.long_amount = (self.balance_amount / long_price)
        self.balance_amount = 0
        self.long_price = long_price
        self.long_unit = symbol
        self.buys.append([symbol, time, long_price, 'long'])
        self.buys.append([symbol, time, long_price, 'previous'])
        self.number_of_buys = len(self.buys)
        print('filled long', long_price, time, '\n\n')

    def buy_short(self, symbol, short_price, atr, time):
        self.balance_amount = self.balance_amount * self.trading_fee_multiplier
        self.short_amount = (self.balance_amount / short_price) 
        self.short_price = short_price
        self.short_unit = symbol
        self.buys.append([symbol, time, short_price, 'short'])
        self.buys.append([symbol, time, short_price, 'previous'])
        self.number_of_buys = len(self.buys)
        self.sl = short_price + (env.slm*atr)
        print('filled short', short_price, time, '\n\n')

    def sell_long(self, sell_price, time, ratio):
        self.balance_amount += (self.long_amount * ratio * sell_price) * self.trading_fee_multiplier
        self.long_amount = self.long_amount * (1-ratio)
        if self.long_amount == 0:
            self.long_unit = None
            self.history.append(self.balance_amount)
        self.sells.append([symbol, time, sell_price])
        self.balance_unit = 'BUSD'
    
    def sell_short(self, sell_price, time):
        self.balance_amount = (self.balance_amount + (self.short_amount * ((self.short_price*self.trading_fee_multiplier) - sell_price)))
        self.sells.append([symbol, time, sell_price])
        self.short_unit = None
        self.short_amount = 0
    


In [93]:
"""
BUY:
    Long: 
        1. Check if symbol crosses under 30 on RSI -> Buy

    Short:
        1. Check if symbol crosses over 70 on RSI

"""

'\nBUY:\n    Long: \n        1. Check if symbol crosses under 30 on RSI -> Buy\n\n    Short:\n        1. Check if symbol crosses over 70 on RSI\n\n'

In [94]:
env = TradingEnv(balance_amount=100, balance_unit='BUSD', symbols=symbols)
for i in range(len(df)):    
    print(env.balance_amount, env.balance_unit, env.number_of_buys)
    if env.long_unit == None and env.short_unit == None:  # Want to buy
        for symbol in symbols:

            ######## LONG           (lower + ((upper-lower)*multiplier))
            if df[f'{symbol}_RSI'].iloc[i] <= 30:
                env.buy_long(symbol, long_price=df[f'{symbol}-USD_Close'].iloc[i],time=df['OpenTime'].iloc[i])
                break
    

            ######## SHORT
            # if df[f'{symbol}_RSI'].iloc[i] >= 70:
            #     env.buy_short(symbol, short_price=df[f'{symbol}-USD_Close'].iloc[i], atr=df[f'{symbol}_ATR'].iloc[i], time=df['OpenTime'].iloc[i])
            #     break



    ####################################################################################################################

    ############## Want to sell long position
    elif env.long_unit != None:
        # Sell long for profit
        if df[f'{symbol}_RSI'].iloc[i] >= 70 and env.crossed_up == False:
            print('sell long 70 up', df[f'{env.long_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.sell_long(sell_price=df[f'{env.long_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i], ratio=.5)
            env.crossed_up = True
        
        if df[f'{symbol}_RSI'].iloc[i] >= 85 and env.crossed_up == True:
            print('sell long 80', df[f'{env.long_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.sell_long(sell_price=df[f'{env.long_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i], ratio=1)
            env.crossed_up = False
            continue

        elif df[f'{symbol}_RSI'].iloc[i] <= 70 and env.crossed_up == True:
            print('sell long 70 down', df[f'{env.long_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.sell_long(sell_price=df[f'{env.long_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i], ratio=1)
            env.crossed_up = False
            continue

    ############## Want to sell short position
    
    # else:
    #     print(env.short_amount)
    #     # Sell short for profit
    #     if env.cross_down[env.short_unit] == True and df[f'{env.short_unit}-USD_Close'].iloc[i] >= (df[f'{symbol}_Lower_Band'].iloc[i] + ((df[f'{symbol}_Upper_Band'].iloc[i]-df[f'{symbol}_Lower_Band'].iloc[i]) * env.band_stop_run)):
    #         print('sell short', df[f'{env.short_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
    #         env.wins += 1
    #         env.sell_short(sell_price=df[f'{env.short_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
    #         env.reset_crosses(symbols)
    #         continue
        
    #     # Sell short at stop loss
    #     elif df[f'{env.short_unit}-USD_Close'].iloc[i] >= env.sl:
    #         print('Stop loss hit: sell short', df[f'{env.short_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
    #         env.losses += 1
    #         env.sell_short(sell_price=df[f'{env.short_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
    #         env.reset_crosses(symbols)
    #         continue
        
    #     # Sell short at break-even  
    #     if env.sma_cross_down[env.short_unit] == True and df[f'{env.short_unit}-USD_High'].iloc[i] >= env.short_price:
    #         print('Break even hit: sell short', env.short_price, df['OpenTime'].iloc[i], '\n\n')
    #         env.ties += 1
    #         env.sell_short(sell_price=env.short_price, time=df['OpenTime'].iloc[i])
    #         env.reset_crosses(symbols)
    #         continue

    #     # Check if SMA is crossed down
    #     if df[f'{env.short_unit}-USD_Low'].iloc[i] <= df[f'{env.short_unit}_SMA'].iloc[i]:
    #         print('SMA short crossed')
    #         env.sma_cross_down[env.short_unit] = True

    #     # Check if lower band is crossed
    #     if df[f'{env.short_unit}-USD_Low'].iloc[i] <= df[f'{env.short_unit}_Lower_Band'].iloc[i]:
    #         print('short crossed')
    #         env.cross_down[env.short_unit] = True
    




if env.long_unit != None:
    env.sell_long(df[f'{env.long_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1], 1)
if env.short_unit != None:
    env.sell_short(df[f'{env.short_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
print(env.balance_amount, env.balance_unit, env.number_of_buys, env.wins, env.ties, env.losses)

100 BUSD 0
filled long 1110.12 2021-01-13 22:15:00 


0 BUSD 2
0 BUSD 2
0 BUSD 2
0 BUSD 2
0 BUSD 2
0 BUSD 2
0 BUSD 2
0 BUSD 2
0 BUSD 2
0 BUSD 2
0 BUSD 2
0 BUSD 2
0 BUSD 2
0 BUSD 2
0 BUSD 2
0 BUSD 2
sell long 70 up 1159.56 2021-01-14 02:15:00 


52.16935262705383 BUSD 2
52.16935262705383 BUSD 2
sell long 70 down 1150.1 2021-01-14 02:45:00 


103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.91309374987163 BUSD 2
103.913

In [95]:

data = pd.DataFrame([(i[1] for i in env.buys), (j[2] for j in env.buys), (a[3] for a in env.buys), (k[1] for k in env.sells), (l[2] for l in env.sells)])
data

,0,1,2,3,4,5,6,7,8,9,...,228,229,230,231,232,233,234,235,236,237
0,2021-01-13 22:15:00,2021-01-13 22:15:00,2021-01-15 10:45:00,2021-01-15 10:45:00,2021-01-17 22:15:00,2021-01-17 22:15:00,2021-01-19 14:30:00,2021-01-19 14:30:00,2021-01-20 23:45:00,2021-01-20 23:45:00,...,2022-01-04 13:45:00,2022-01-04 13:45:00,2022-01-10 06:00:00,2022-01-10 06:00:00,2022-01-13 12:15:00,2022-01-13 12:15:00,2022-01-15 07:30:00,2022-01-15 07:30:00,2022-01-15 21:45:00,2022-01-15 21:45:00
1,1110.12,1110.12,1143.08,1143.08,1201.2,1201.2,1345.68,1345.68,1288.65,1288.65,...,3771.05,3771.05,3114.74,3114.74,3267.4,3267.4,3273.31,3273.31,3302.73,3302.73
2,long,previous,long,previous,long,previous,long,previous,long,previous,...,long,previous,long,previous,long,previous,long,previous,long,previous
3,2021-01-14 02:15:00,2021-01-14 02:45:00,2021-01-16 03:30:00,2021-01-16 03:45:00,2021-01-18 20:00:00,2021-01-18 20:30:00,2021-01-20 18:45:00,2021-01-20 19:00:00,2021-01-23 22:45:00,2021-01-24 00:00:00,...,2022-01-09 14:15:00,2022-01-09 15:15:00,2022-01-11 11:30:00,2022-01-11 11:45:00,2022-01-14 16:45:00,2022-01-14 17:15:00,2022-01-15 10:30:00,2022-01-15 11:00:00,2022-01-18 19:14:59.999,None
4,1159.56,1150.1,1219.89,1218.43,1274.7,1327.94,1379.24,1369.71,1294.62,1331.65,...,3200.53,3198.69,3228.22,3218.0,3329.67,3319.99,3357.05,3357.89,3157.89,None


In [96]:
env.history

[100,
 103.91309374987163,
 110.70736111444756,
 119.80292580073768,
 122.23191510046087,
 124.41741099786128,
 122.05162195061493,
 127.03440356403212,
 138.515892416898,
 144.63808404244998,
 149.48810091785168,
 154.7424397915694,
 155.00169198581355,
 161.43512847110958,
 141.6091695477268,
 127.0887507976754,
 132.84069801443772,
 136.30783733156613,
 145.1490126690171,
 154.0012397116089,
 156.21200317975743,
 162.2191605092495,
 157.39717374963323,
 160.7286127315677,
 159.44334572006466,
 154.9910390448175,
 152.57676576949953,
 157.4300969449418,
 162.36338790349964,
 164.5564446364684,
 169.8350361827652,
 167.73652917417866,
 174.75141680471802,
 159.74423474432135,
 163.7602563640176,
 157.06231872341428,
 171.4846692104006,
 180.16321303661442,
 185.98487482679374,
 193.051774388736,
 204.61498249779342,
 213.67963157729605,
 204.00408497222912,
 191.29007890132254,
 169.34520964961666,
 153.24213385231934,
 167.35803311411433,
 154.51136002623886,
 160.49552208508447,
 16